# Test the code before implemented

In [1]:
import json
import pandas as pd
import random
import re
import requests
import time
import numpy as np
import cloudscraper


SCHEMA = {
    "condition":"object",
    "created_at":"datetime64[ns]",
    "id":"object",
    "max_quantity":"object",
    "merchant_return_insurance":"bool",
    "min_quantity":"Int64",
    "name":"object",
    "price":"Int64",
    "rush_delivery":"bool",
    "sku_id":"object",
    "state":"object",
    "stock":"Int64",
    "weight":"Int64",
    "category_url":"object",
    "rating_average_rate":"float64",
    "rating_user_count":"Int64",
    "sla_type":"object",
    "sla_value":"object",
    "specs_brand":"object",
    "stats_interest_count":"Int64",
    "stats_sold_count":"Int64",
    "stats_view_count":"Int64",
    "stats_waiting_payment_count":"float64",
    "store_address_city":"object",
    "store_address_province":"object",
    "store_brand_seller":"bool",
    "store_delivery_time":"object",
    "store_id":"object",
    "store_description":"object",
    "store_level_name":"object",
    "store_name":"object",
    "store_premium_level":"object",
    "store_premium_top_seller":"bool",
    "store_rejection_recent_transactions":"Int64",
    "store_rejection_rejected":"Int64",
    "store_reviews_negative":"Int64",
    "store_reviews_positive":"Int64",
    "store_sla_type":"object",
    "store_sla_value":"Int64",
    "store_subscriber_amount":"Int64",
    "store_url":"object",
    "warranty_cheapest":"bool",
    "deal_applied_date":"datetime64[ns]",
    "deal_discount_price":"Int64",
    "deal_expired_date":"datetime64[ns]",
    "deal_original_price":"Int64",
    "deal_percentage":"Int64",
    "url":"object",
    "timestamp":"datetime64[ns]"
}

ADD_COL_TYPE = {
    'datetime': ['timestamp','created_at','deal_applied_date','deal_expired_date'],
    'bool_str': ['merchant_return_insurance', 'rush_delivery', 'store_brand_seller', 'store_premium_top_seller', 'warranty_cheapest'],
}

In [11]:
def get_token():
    res = requests.get("https://bukalapak.com",
                       headers={
                           "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0",
                       })
    return re.search("\"access_token\":\"(.*?)\"",res.text).group()[16:-1]
    


get_token()

'qDQdYe7xvbHCHgLLmqaViITN4sHbVhE7oZeKd62iLyLoAw'

In [16]:
print (get_token())

qDQdYe7xvbHCHgLLmqaViITN4sHbVhE7oZeKd62iLyLoAw


In [17]:
def get_scrape (params, get_token, page = 2, URL = "https://api.bukalapak.com/multistrategy-products"):
    DF = []
    index = 1
    while index <= page:
        payload = {
            "offset" : ((index-1)*30),
            "page" : index,
            "access_token" : get_token(),
            **params
        }

        headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0",
            }
    
        scrape = requests.get (url=URL, params = payload, headers=headers)
        sleep_time = random.randint(10, 50)
        time.sleep(sleep_time / 1000)

    
        return print (scrape.status_code)

In [18]:
params = {
    "prambanan_override" : "true",
    "category_id" : 2308,
    "sort" : "bestselling",
    "limit" : 30,
    "facet" : "true",
    # "brand": "true"
}
get_scrape(params, get_token)

200


In [10]:
params = {
    "offset" : 0,
    "page" : 1,
    "access_token" : "Yam1Z2lIBLKLlYHEWWXhKheQIS46-zXOvuIMvy3MxVRYzA",
    "prambanan_override" : "true",
    "category_id" : 3263,
    "sort" : "bestselling",
    "limit" : 30,
    "facet" : "true",
    # "brand": "true"
}

headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0",
            }

scraper = cloudscraper.create_scraper()

scrape = scraper.get("https://api.bukalapak.com/multistrategy-products", params = params, headers = headers)

scrape.status_code

401

In [1]:
import json
import pandas as pd
import random
import re
import requests
import time
import numpy as np

In [2]:
def get_token():
    res = requests.get("https://bukalapak.com",
                       headers={
                           "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0",
                       })
    return re.search("\"access_token\":\"(.*?)\"",res.text).group()[16:-1] # looking for token access

In [3]:
def get_scrape (params, get_token, page = 10, URL = "https://api.bukalapak.com/multistrategy-products"):
    DF = []
    index = 1
    while index <= page:
        payload = {
            "offset" : ((index-1)*30),
            "page" : index,
            "access_token" : get_token(),
            **params
        }

        headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0",
            }
    
        scrape = requests.get (URL, params = payload, headers=headers)
        sleep_time = random.randint(10, 50)
        time.sleep(sleep_time / 1000)

        if scrape.status_code == 200:
            json_file = scrape.json()
            df_1 = pd.json_normalize(json_file)
            df_2 = json.loads(pd.Series.to_json(df_1["data"]))
            df_3 = pd.json_normalize(df_2, record_path="0")
            DF.append(df_3)
        if scrape.status_code != 200:
            raise ValueError("Error returned: {}".format(res.status_code))
        index = index+1
    df_scraper = pd.concat(DF, ignore_index=True)
    return df_scraper

In [4]:
params = {
    "prambanan_override" : "true",
    "category_id" : 3263,
    "sort" : "bestselling",
    "limit" : 30,
    "facet" : "true",
    # "brand": "true"
}
df_scraper = get_scrape(params, get_token)

In [5]:
df_scraper

,active,assurance,available_countries,condition,couriers,created_at,default_catalog,description,digital_product,discount_percentage,...,warranty.cheapest,deal.applied_date,deal.discount_price,deal.expired_date,deal.original_price,deal.percentage,specs.halal,store.closing.closed_reason,store.closing.end_date,store.closing.start_date
0,True,False,[],Baru,[],2021-11-18T15:33:56Z,None,NEO COFFEE 3IN1 INSTANT COFFEE <br><br/><br><b...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,False,[],Baru,[],2022-01-24T10:02:58Z,None,"<p>Weekly Roast, Dijamin Fresh! sehingga bisa ...",False,10,...,False,2022-02-20T17:00:00Z,57500.0,2022-12-31T16:59:59Z,64000.0,10.0,NaN,NaN,NaN,NaN
2,True,False,[],Baru,[],2022-01-18T01:40:15Z,None,Isi 1 pack 10pcs<br><br/>Berat 1 pack 400g<br>...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,False,[],Baru,[],2022-04-19T04:18:48Z,None,<p>PRIVASI TERJAGA<br>FREE PACKING DUS &amp; B...,False,22,...,False,2022-06-16T17:00:00Z,390000.0,2022-11-30T16:59:59Z,500000.0,22.0,[],NaN,NaN,NaN
4,True,False,[],Baru,[],2021-11-18T15:34:21Z,None,TORABIKA CAPPUCCINO <br><br/>1RENCENG = 10SACH...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,True,False,[],Baru,[],2022-03-28T09:41:30Z,None,Biji kopi bulat kingkong sudah di sangrai <br>...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,True,False,[],Baru,[],2022-02-21T10:16:11Z,None,<p>Terbuat Dari Biji Kopi Terpilih Yang Dipros...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,True,False,[],Baru,[],2021-02-23T03:47:55Z,None,<p>JJ Royal Coffee Java Monk Robusta.</p><p>An...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
298,True,False,[],Baru,[],2021-02-23T04:43:27Z,None,<p>Jenis Kopi : Kopi bubuk/Ground<br>Taste Kop...,False,0,...,False,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
